In [1]:
from guppylang.std.builtins import comptime
from guppylang.std.quantum import qubit, measure_array
from hugr.qsystem.result import QsysResult
from guppylang.std.builtins import result, array, exit
from guppylang import guppy
from selene_sim import build, Quest, MetricStore

from trunctaylor.qtmlib.circuits.lcu import LCUMultiplexorBox
from pytket.passes import AutoRebase
from pytket import OpType
from pytket.passes import DecomposeBoxes
from pytket.circuit import StatePreparationBox
from pytket.circuit import Circuit, CircBox

from collections import defaultdict

import pandas as pd
import numpy as np
import scipy.special as sp

from trunctaylor.operators import ising_model

### 1D Ising model Hamiltonian

In [2]:
n_state = 4
H = ising_model(n_qubits=n_state, j=1.0, h=0.5)

We construct the LCU for
$H_k= (-i)^k \sum_{\ell}^{L}\alpha_\ell H_\ell$
using multiplexor gate synthesis. We then construct a controlled-$H_k$ for the Select oracle of the time evolution operator.

In [3]:
H_lcu = LCUMultiplexorBox(H, n_state)
H_norm = H_lcu.l1_norm

n_prep = H_lcu.n_prepare_qubits

H_lcu0 = LCUMultiplexorBox((1 / H_norm) * H, n_state) #(-i)^k = 1
H_lcu1 = LCUMultiplexorBox((-1j / H_norm) * H, n_state) #(-i)^k = -i
H_lcu2 = LCUMultiplexorBox((-1 / H_norm) * H, n_state) #(-i)^k = -1
H_lcu3 = LCUMultiplexorBox((1j / H_norm) * H, n_state) #(-i)^k = i

controlled_lcu0 = H_lcu0.qcontrol(1)
circ = controlled_lcu0.get_circuit()
DecomposeBoxes().apply(circ)
rebase = AutoRebase({OpType.CX, OpType.Rz, OpType.H, OpType.CCX})
rebase.apply(circ)
qlibs_controlled_lcu0 = guppy.load_pytket(
    "qlibs_controlled_lcu0", circ, use_arrays=True
)

controlled_lcu1 = H_lcu1.qcontrol(1)
circ = controlled_lcu1.get_circuit()
DecomposeBoxes().apply(circ)
rebase = AutoRebase({OpType.CX, OpType.Rz, OpType.H, OpType.CCX})
rebase.apply(circ)
qlibs_controlled_lcu1 = guppy.load_pytket(
    "qlibs_controlled_lcu1", circ, use_arrays=True
)

controlled_lcu2 = H_lcu2.qcontrol(1)
circ = controlled_lcu2.get_circuit()
DecomposeBoxes().apply(circ)
rebase = AutoRebase({OpType.CX, OpType.Rz, OpType.H, OpType.CCX})
rebase.apply(circ)
qlibs_controlled_lcu2 = guppy.load_pytket(
    "qlibs_controlled_lcu2", circ, use_arrays=True
)

controlled_lcu3 = H_lcu3.qcontrol(1)
circ = controlled_lcu3.get_circuit()
DecomposeBoxes().apply(circ)
rebase = AutoRebase({OpType.CX, OpType.Rz, OpType.H, OpType.CCX})
rebase.apply(circ)
qlibs_controlled_lcu3 = guppy.load_pytket(
    "qlibs_controlled_lcu3", circ, use_arrays=True
)

@guppy.comptime
def ctrl_lcu0(
    control: qubit,
    prep: array[qubit, comptime(n_prep)],
    state: array[qubit, comptime(n_state)],
) -> None:
    return qlibs_controlled_lcu0([control], prep, state)

@guppy.comptime
def ctrl_lcu1(
    control: qubit,
    prep: array[qubit, comptime(n_prep)],
    state: array[qubit, comptime(n_state)],
) -> None:
    return qlibs_controlled_lcu1([control], prep, state)

@guppy.comptime
def ctrl_lcu2(
    control: qubit,
    prep: array[qubit, comptime(n_prep)],
    state: array[qubit, comptime(n_state)],
) -> None:
    return qlibs_controlled_lcu2([control], prep, state)

@guppy.comptime
def ctrl_lcu3(
    control: qubit,
    prep: array[qubit, comptime(n_prep)],
    state: array[qubit, comptime(n_state)],
) -> None:
    return qlibs_controlled_lcu3([control], prep, state)


Prepare the  Taylor expansion coefficients 
$\widetilde{\beta}_k = \frac{(\tau \left\|\alpha\right\|_{1})^k}{k!}$ for $\sqrt{\frac{\widetilde{\beta}_k}{\left\|\widetilde{\beta}\right\|_1}}$.

We set $K=3$, $\tau=0.05$.

In [4]:
K= 3
tau = 0.05
def beta_coeff(tau: float, alpha: float, K: int):
    beta_k = np.zeros(K + 1)
    beta_k[0] = 1.0
    for k in range(1, K + 1):
        beta_k[k] = ((tau * alpha) ** k) / sp.factorial(k)
    return beta_k


def PrepareUnaryBox(beta_sqrt):
    theta = np.zeros(len(beta_sqrt) - 1)
    theta[0] = np.arccos(beta_sqrt[0]) * 2 / np.pi

    for i in range(1, len(beta_sqrt) - 1):
        theta[i] = (
            np.arccos(beta_sqrt[i] / np.sqrt(1 - np.sum(beta_sqrt[:i] ** 2)))
            * 2
            / np.pi
        )

    circ = Circuit(len(beta_sqrt) - 1, name="PrepUnary")
    circ.Ry(theta[0], 0)
    for i in range(1, len(beta_sqrt) - 1):
        circ.CRy(theta[i], i - 1, i)

    DecomposeBoxes().apply(circ)
    rebase = AutoRebase({OpType.CX, OpType.Rz, OpType.H, OpType.CCX})
    rebase.apply(circ)
    prep_unary_box = CircBox(circ)

    return prep_unary_box

betas = beta_coeff(tau=tau, alpha=H_norm, K=K)
betas_rescaled = np.sqrt(betas / np.sum(np.abs(betas)))

exp_prep_box = PrepareUnaryBox(betas_rescaled)
circ = exp_prep_box.get_circuit()
DecomposeBoxes().apply(circ)
rebase = AutoRebase({OpType.CX, OpType.Rz, OpType.H, OpType.CCX})
rebase.apply(circ)
exp_prep = guppy.load_pytket("exp_prepare", circ, use_arrays=True)
exp_unprep = guppy.load_pytket("exp_unprepare", circ.dagger(), use_arrays=True)

Putting all together, we implement the LCU with deferred measurements for the time evolution operator.
We run $1000$ shots for now.

In [5]:
n_shots = 1000

@guppy
def allocate() -> array[array[qubit, comptime(n_prep)], comptime(K)]:
    prep_qubits = array(
        array(qubit() for _ in range(comptime(n_prep))) for _ in range(comptime(K))
    )
    return prep_qubits

@guppy
def main() -> None:
    """Main function to run the multiplexor LCU circuit."""
    exp_prep_qubits = array(qubit() for _ in range(comptime(K)))
    prep_array = allocate()

    state_qubits = array(qubit() for _ in range(comptime(n_state)))
    # The pytket function only acts on arrays
    exp_prep(exp_prep_qubits)

    for kk_index in range(comptime(K)):
        if kk_index + 1 % 4 == 1:
            ctrl_lcu1(exp_prep_qubits[kk_index], prep_array[kk_index], state_qubits)
        elif kk_index + 1 % 4 == 2:
            ctrl_lcu2(exp_prep_qubits[kk_index], prep_array[kk_index], state_qubits)
        elif kk_index + 1 % 4 == 3:
            ctrl_lcu3(exp_prep_qubits[kk_index], prep_array[kk_index], state_qubits)
        else:
            ctrl_lcu0(exp_prep_qubits[kk_index], prep_array[kk_index], state_qubits)

    exp_unprep(exp_prep_qubits)

    outcome = array(measure_array(ar) for ar in prep_array)
    for measured in outcome:
        for b in measured:
            if b:
                exit("circuit failed", 1)
    outcome = measure_array(exp_prep_qubits)
    for b in outcome:
        if b:
            exit("circuit failed", 1)

    result("c", measure_array(state_qubits))

compiled_hugr = main.compile()
metric_store = MetricStore()

runner = build(compiled_hugr)
shots = QsysResult(
        runner.run_shots(
            Quest(random_seed=17),
            event_hook=metric_store,
            n_qubits=K + (K * n_prep) + n_state,
            n_shots=n_shots,
            verbose=False,
        )
    )

Final state and success probability

In [6]:
shots_counts = shots.register_counts()["c"]
success_prob = sum(shots_counts.values()) / n_shots

print(f'Final state: {shots_counts}')
print(f'Success probability: {success_prob}')

Final state: Counter({'0000': 625, '0100': 1, '0001': 1})
Success probability: 0.627


Circuit resources used

In [7]:
metrics = metric_store.shots
data_by_category = defaultdict(lambda: defaultdict(list))

for shot in metrics:
    for category, metric_dict in shot.items():
        for metric, value in metric_dict.items():
            data_by_category[category][metric].append(value)

headers = ["Metric"] + [f"Shot {i}" for i in range(len(metrics))]

df_user_program = pd.DataFrame(
    [[k] + v[:] for k, v in data_by_category["user_program"].items()],
    columns=headers,
)
df_user_program = df_user_program.set_index("Metric").transpose()

df_post_runtime = pd.DataFrame(
    [[k] + v[:] for k, v in data_by_category["post_runtime"].items()],
    columns=headers,
)
df_post_runtime = df_post_runtime.set_index("Metric").transpose()


print(f'Average metrics (over shots):\n{df_post_runtime.mean()}')

Average metrics (over shots):
Metric
custom_op_batch_count            0.000
custom_op_individual_count       0.000
measure_batch_count             14.112
measure_individual_count        14.112
reset_batch_count               16.000
reset_individual_count          16.000
rxy_batch_count               1384.000
rxy_individual_count          1384.000
rz_batch_count                2199.000
rz_individual_count           2199.000
rzz_batch_count                458.000
rzz_individual_count           458.000
total_duration_ns                0.000
dtype: float64


Classical results

In [8]:
H_mat = H.to_sparse_matrix().toarray()
state = np.zeros(2**n_state)
state[0] = 1.0

p_analytical = []

for K in range(1, 8):
    betas = beta_coeff(tau=0.05, alpha=H_norm, K=K)
    exp_ihht = betas[0] * np.identity(2**n_state)
    for k in range(1, K + 1):
        Hk = H_mat / H_norm
        for kk in range(k - 1):
            Hk = Hk @ H_mat / H_norm
        exp_ihht = exp_ihht + (-1j) ** k * betas[k] * Hk
    
    statef = exp_ihht @ state

    p_analytical.append(statef.conj().T @ statef / ((sum(betas)) ** 2))
    print(f'Success probability (analytical), K={K}: {statef.conj().T @ statef / ((sum(betas)) ** 2)}')

Success probability (analytical), K=1: (0.6559999999999999+0j)
Success probability (analytical), K=2: (0.6092673408685306+0j)
Success probability (analytical), K=3: (0.6066575788750417+0j)
Success probability (analytical), K=4: (0.6065385140258427+0j)
Success probability (analytical), K=5: (0.6065310248629526+0j)
Success probability (analytical), K=6: (0.6065306716397784+0j)
Success probability (analytical), K=7: (0.6065306600635357+0j)
